In [18]:
import numpy as np
import os
import torch
from sklearn.linear_model import Lasso
from torch_geometric.data import Data, Dataset, DataLoader
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GINConv, global_add_pool

In [19]:
# -----------------------------
# Parameters
# -----------------------------
data_folder = "data"  # folder with all .1D files
alpha = 0.1  # L1 sparsity penalty
beta = 0.01  # graph smoothness penalty

In [20]:
subject_time_series = {}

for file_name in file_list:

    subject_id = "_".join(file_name.split("_")[:2])  
    file_path = os.path.join(data_folder, file_name)
    ts = np.loadtxt(file_path)
    subject_time_series[subject_id] = ts
    print(f"Loaded {subject_id}, shape: {ts.shape}")

print(f"Total subjects loaded: {len(subject_time_series)}")  

Loaded KKI_0050822, shape: (152, 200)
Loaded NYU_0050957, shape: (176, 200)
Loaded NYU_0050967, shape: (176, 200)
Loaded NYU_0050970, shape: (176, 200)
Loaded NYU_0050974, shape: (176, 200)
Loaded NYU_0050976, shape: (176, 200)
Loaded NYU_0050983, shape: (176, 200)
Loaded NYU_0050987, shape: (176, 200)
Loaded NYU_0050991, shape: (176, 200)
Loaded NYU_0050993, shape: (176, 200)
Loaded NYU_0051002, shape: (176, 200)
Loaded NYU_0051006, shape: (176, 200)
Loaded NYU_0051013, shape: (176, 200)
Loaded NYU_0051021, shape: (176, 200)
Loaded NYU_0051027, shape: (176, 200)
Loaded NYU_0051032, shape: (176, 200)
Loaded NYU_0051041, shape: (176, 200)
Loaded OHSU_0050147, shape: (78, 200)
Loaded OHSU_0050147, shape: (78, 200)
Total subjects loaded: 18


In [21]:
len(subject_time_series)


18

In [ ]:
# -----------------------------
# Step 2: Compute graph-regularized LASSO for each subject
# -----------------------------
subject_graphs = {}
for subject, X in subject_time_series.items():
    ROIs = X.shape[1]
    W = np.zeros((ROIs, ROIs))
    
    
    L = np.eye(ROIs)
    
    for i in range(ROIs):
        y = X[:, i]
        X_rest = np.delete(X, i, axis=1)
        
       
        lasso = Lasso(alpha=alpha, max_iter=5000)
        lasso.fit(X_rest, y)
        w = lasso.coef_
        
        
        w_full = np.insert(w, i, 0)
        
        
        w_full = w_full - beta * L[i]
        
        W[i, :] = w_full
    
    subject_graphs[subject] = W
    print(f"Computed graph-regularized LASSO for {subject}, shape: {W.shape}")

c:\Users\MehwishSameer\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.083e+01, tolerance: 1.007e+01
  model = cd_fast.enet_coordinate_descent(
c:\Users\MehwishSameer\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 8.715e+00, tolerance: 4.630e+00
  model = cd_fast.enet_coordinate_descent(
c:\Users\MehwishSameer\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of i

Computed graph-regularized LASSO for KKI_0050822, shape: (200, 200)


c:\Users\MehwishSameer\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 6.622e+01, tolerance: 1.640e+01
  model = cd_fast.enet_coordinate_descent(
c:\Users\MehwishSameer\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.458e+01, tolerance: 1.111e+01
  model = cd_fast.enet_coordinate_descent(
c:\Users\MehwishSameer\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of i

Computed graph-regularized LASSO for NYU_0050957, shape: (200, 200)


c:\Users\MehwishSameer\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.348e+01, tolerance: 6.461e+00
  model = cd_fast.enet_coordinate_descent(
c:\Users\MehwishSameer\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.353e+01, tolerance: 6.994e+00
  model = cd_fast.enet_coordinate_descent(
c:\Users\MehwishSameer\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of i

...

c:\Users\MehwishSameer\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 6.096e+00, tolerance: 5.092e+00
  model = cd_fast.enet_coordinate_descent(
c:\Users\MehwishSameer\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.022e+01, tolerance: 9.269e+00
  model = cd_fast.enet_coordinate_descent(
c:\Users\MehwishSameer\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of i

Computed graph-regularized LASSO for NYU_0051041, shape: (200, 200)


c:\Users\MehwishSameer\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.006e+00, tolerance: 1.403e+00
  model = cd_fast.enet_coordinate_descent(
c:\Users\MehwishSameer\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.210e+00, tolerance: 1.538e+00
  model = cd_fast.enet_coordinate_descent(
c:\Users\MehwishSameer\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of i

Computed graph-regularized LASSO for OHSU_0050147, shape: (200, 200)


c:\Users\MehwishSameer\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.453e+00, tolerance: 7.626e-01
  model = cd_fast.enet_coordinate_descent(
c:\Users\MehwishSameer\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.645e+00, tolerance: 1.597e+00
  model = cd_fast.enet_coordinate_descent(


In [23]:
# -----------------------------
# Step 3: Save all graphs
# -----------------------------
save_folder = "graphs"
os.makedirs(save_folder, exist_ok=True)

for subject, W in subject_graphs.items():
    np.save(os.path.join(save_folder, f"{subject}_graph.npy"), W)

print("All graphs saved to folder:", save_folder)

All graphs saved to folder: graphs


In [24]:
# -----------------------------
# Load graphs
# -----------------------------
graph_folder = "graphs"
graph_files = [f for f in os.listdir(graph_folder) if f.endswith("_graph.npy")]

# Dummy labels (replace with real labels!)
labels_dict = {f.split("_")[0]: np.random.randint(0,2) for f in graph_files}  

In [25]:
# -----------------------------
# Convert to PyG Data objects
# -----------------------------
data_list = []

for f in graph_files:
    subject_id = f.split("_")[0]
    W = np.load(os.path.join(graph_folder, f))  # ROIs x ROIs
    
    # Node features: here we use identity (1 feature per ROI)
    x = torch.eye(W.shape[0], dtype=torch.float)  # ROIs x  ROIs as simple features
    
    # Edge index and edge weights
    # Convert adjacency matrix to edge list format
    src, dst = np.where(W != 0)
    edge_index = torch.tensor(np.vstack([src,dst]), dtype=torch.long)
    edge_attr = torch.tensor(W[src, dst], dtype=torch.float).unsqueeze(1)
    
    # Label
    y = torch.tensor([labels_dict[subject_id]], dtype=torch.long)
    
    # Create PyG Data object
    data = Data(x=x, edge_index=edge_index, edge_attr=edge_attr, y=y)
    data_list.append(data)

print(f"Created {len(data_list)} PyG graph objects.")

batch_size = 4
loader = DataLoader(data_list, batch_size=batch_size, shuffle=True)

Created 21 PyG graph objects.


C:\Users\MehwishSameer\AppData\Local\Temp\ipykernel_23352\450708825.py:29: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  loader = DataLoader(data_list, batch_size=batch_size, shuffle=True)


In [26]:
class GINNet(nn.Module):
    def __init__(self, num_features, hidden_dim=32, num_classes=2):
        super(GINNet, self).__init__()
        
        # MLP for GINConv
        nn1 = nn.Sequential(nn.Linear(num_features, hidden_dim), nn.ReLU(), nn.Linear(hidden_dim, hidden_dim))
        nn2 = nn.Sequential(nn.Linear(hidden_dim, hidden_dim), nn.ReLU(), nn.Linear(hidden_dim, hidden_dim))
        
        self.conv1 = GINConv(nn1)
        self.conv2 = GINConv(nn2)
        self.fc = nn.Linear(hidden_dim, num_classes)
        
    def forward(self, data):
        x, edge_index, edge_attr, batch = data.x, data.edge_index, data.edge_attr, data.batch
        
        x = self.conv1(x, edge_index, edge_attr)
        x = F.relu(x)
        x = self.conv2(x, edge_index, edge_attr)
        x = F.relu(x)
        
        x = global_add_pool(x, batch)
        x = self.fc(x)
        return x

In [27]:
class GINNet(nn.Module):
    def __init__(self, num_features, hidden_dim=32, num_classes=2):
        super(GINNet, self).__init__()
        
        # MLP for GINConv
        nn1 = nn.Sequential(
            nn.Linear(num_features, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim)
        )
        nn2 = nn.Sequential(
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim)
        )
        
        self.conv1 = GINConv(nn1)
        self.conv2 = GINConv(nn2)
        self.fc = nn.Linear(hidden_dim, num_classes)
        
    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch  # removed edge_attr
        
        x = self.conv1(x, edge_index)  # removed edge_attr
        x = F.relu(x)
        x = self.conv2(x, edge_index)  # removed edge_attr
        x = F.relu(x)
        
        x = global_add_pool(x, batch)
        x = self.fc(x)
        return x

# -----------------------------
# Initialize model
# -----------------------------
num_features = data_list[0].x.shape[1]
model = GINNet(num_features)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = nn.CrossEntropyLoss()

In [28]:
for epoch in range(50):
    model.train()
    total_loss = 0
    for batch in loader:
        optimizer.zero_grad()
        out = model(batch)
        loss = criterion(out, batch.y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch+1}, Loss: {total_loss/len(loader):.4f}")

Epoch 1, Loss: 34.8997
Epoch 2, Loss: 0.0000
Epoch 3, Loss: 0.0000
Epoch 4, Loss: 0.0000
Epoch 5, Loss: 0.0000
Epoch 6, Loss: 0.0000
Epoch 7, Loss: 0.0000
Epoch 8, Loss: 0.0000
Epoch 9, Loss: 0.0000
Epoch 10, Loss: 0.0000
Epoch 11, Loss: 0.0000
Epoch 12, Loss: 0.0000
Epoch 13, Loss: 0.0000
Epoch 14, Loss: 0.0000
Epoch 15, Loss: 0.0000
Epoch 16, Loss: 0.0000
Epoch 17, Loss: 0.0000
Epoch 18, Loss: 0.0000
Epoch 19, Loss: 0.0000
Epoch 20, Loss: 0.0000
Epoch 21, Loss: 0.0000
Epoch 22, Loss: 0.0000
Epoch 23, Loss: 0.0000
Epoch 24, Loss: 0.0000
Epoch 25, Loss: 0.0000
Epoch 26, Loss: 0.0000
Epoch 27, Loss: 0.0000
Epoch 28, Loss: 0.0000
Epoch 29, Loss: 0.0000
Epoch 30, Loss: 0.0000
Epoch 31, Loss: 0.0000
Epoch 32, Loss: 0.0000
Epoch 33, Loss: 0.0000
Epoch 34, Loss: 0.0000
Epoch 35, Loss: 0.0000
Epoch 36, Loss: 0.0000
Epoch 37, Loss: 0.0000
Epoch 38, Loss: 0.0000
Epoch 39, Loss: 0.0000
Epoch 40, Loss: 0.0000
Epoch 41, Loss: 0.0000
Epoch 42, Loss: 0.0000
Epoch 43, Loss: 0.0000
Epoch 44, Loss: 0.0

In [29]:
model.eval()  

GINNet(
  (conv1): GINConv(nn=Sequential(
    (0): Linear(in_features=200, out_features=32, bias=True)
    (1): ReLU()
    (2): Linear(in_features=32, out_features=32, bias=True)
  ))
  (conv2): GINConv(nn=Sequential(
    (0): Linear(in_features=32, out_features=32, bias=True)
    (1): ReLU()
    (2): Linear(in_features=32, out_features=32, bias=True)
  ))
  (fc): Linear(in_features=32, out_features=2, bias=True)
)

In [30]:
all_preds = []
with torch.no_grad():
    for data in data_list:
        data = data.to('cpu')  # or 'cuda' if GPU
        out = model(data)
        pred_class = out.argmax(dim=1)
        all_preds.append(pred_class.item())

print("Predictions for all subjects:", all_preds)


Predictions for all subjects: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [31]:
from nilearn.datasets import fetch_abide_pcp

dataset = fetch_abide_pcp(data_dir="your_download_folder", pipeline="cpac",
                           band_pass_filtering=True, global_signal_regression=True,
                           derivatives=["rois_cc200"])
phenotype_df = dataset["phenotypic"]
print(phenotype_df.head())


[fetch_abide_pcp] Added README.md to your_download_folder

[fetch_abide_pcp] Dataset created in your_download_folder\ABIDE_pcp

[fetch_abide_pcp] Downloading data from 
https://s3.amazonaws.com/fcp-indi/data/Projects/ABIDE_Initiative/Phenotypic_V1_0b_preprocessed1.csv ...

[fetch_abide_pcp] Downloaded 319488 of 449443 bytes (71.1%%,    0.4s remaining)

[fetch_abide_pcp]  ...done. (2 seconds, 0 min)

...

[fetch_abide_pcp] Downloaded 57344 of 231019 bytes (24.8%%,    3.3s remaining)

[fetch_abide_pcp] Downloaded 122880 of 231019 bytes (53.2%%,    2.0s remaining)

[fetch_abide_pcp] Downloaded 204800 of 231019 bytes (88.7%%,    0.4s remaining)

[fetch_abide_pcp]  ...done. (4 seconds, 0 min)

   i  Unnamed: 0  SUB_ID  X  subject SITE_ID       FILE_ID  DX_GROUP  \
1  1           2   50003  2    50003    PITT  Pitt_0050003         1   
2  2           3   50004  3    50004    PITT  Pitt_0050004         1   
3  3           4   50005  4    50005    PITT  Pitt_0050005         1   
4  4           5   50006  5    50006    PITT  Pitt_0050006         1   
5  5           6   50007  6    50007    PITT  Pitt_0050007         1   

   DSM_IV_TR  AGE_AT_SCAN  ...  qc_notes_rater_1 qc_anat_rater_2  \
1          1        24.45  ...               NaN              OK   
2          1        19.09  ...               NaN              OK   
3          1        13.73  ...               NaN              OK   
4          1        13.37  ...               NaN              OK   
5          1        17.78  ...               NaN              OK   

   qc_anat_notes_rater_2  qc_func_rater_2       qc_func_notes_rater_2  \
1                    NaN               OK                         NaN   
2           

In [37]:
true_labels = []
for subject in subject_time_series.keys():
    
    row = phenotype_df[phenotype_df['FILE_ID'] == subject]
    
    if not row.empty:
        
        label = int(row['DX_GROUP'].values[0])
        true_labels.append(label)
    else:
        print(f"{subject} not found in phenotypic file")


In [40]:
all_preds = []

# Ensure you only iterate over the same subjects
for data in data_list[:len(true_labels)]:  # slice to match true_labels
    data = data.to('cpu')  # or 'cuda'
    out = model(data)
    pred_class = out.argmax(dim=1)
    all_preds.append(pred_class.item())

from sklearn.metrics import accuracy_score
accuracy = accuracy_score(true_labels, all_preds)
print(f"Accuracy on {len(true_labels)} subjects: {accuracy*100:.2f}%")


Accuracy on 18 subjects: 88.89%
